# 系列データのモデル化-リカレントニューラルネットワーク

本章ではリカレントニューラルネットワーク(RNN:Recurrent Neural Network)に焦点を合わせ，系列データのモデル化への応用と，系列データの一種である時系列データを取り上げる．本章では，次の項目を取り上げる．

- 系列データの概要
- RNN：シーケンスモデルの構築
- **長短期記憶**(LSTM)
- **T-BPTT**(Truncated Backpropagation Through Time)
- シーケンスモデルを構築するためのTensorFlowでの多層RNNの実装
- プロジェクト1：RNNによるIMDb映画レビューデータセットの感情分析
- プロジェクト2：シェイクスピアの『ハムレット』を使って文字レベルの言語モデルをRNNモデルとして構築
- 勾配の発散を回避するための勾配刈り込みの使用

## 系列データ

RNNを説明するにあたってまずは系列データの性質から見ていく．

より一般的には系列データは**シーケンス**(sequence)と呼ばれる．
ここでは，系列データのユニークな特性を調べることで，他の種類のデータとどのように異なるのかを明らかにする.

### 系列データのモデル化

系列データ(シーケンス)と他の種類のデータとの違いは，シーケンスでは要素が特定の順序で並んでいて，互いに無関係ではないことにある．

教師あり機械学習の典型的な機械学習アルゴリズムでは，入力データが**独立同分布**(Independent and Identically Distributed:IID)であることが前提となる．

例えば，$n$個のデータサンプル$x^{(1)},x^{(2)},\cdots,x^{(n)}$がある場合，このデータを機械学習のトレーニングに使用するときの順序は問題ではない．

しかし，シーケンスを扱うときには，この前提は成り立たなくなる．当然ながら，順序は重要だからである．

### 系列データを表現する

入力データでは，シーケンスの要素が互いに依存する順序で並んでいることが裏付けられたとする．

本章では，シーケンスを$(x^{(1)},x^{(2)},\cdots,x^{(T)})$で表すことにする．上付き文字はインスタンスの順序を表しており，シーケンスの長さは$T$である．

さて，シーケンスといえば，時系列データである．時系列データでは，各サンプル点$x^{(t)}$が特定の時間$t$に属している．

多層パーセプトロン(MLP)や畳み込みニューラルネットワーク(CNN)と言った標準のニューラルネットワークモデルは，入力サンプルの**順序**を処理できない．

直感的に言えるのは，そうしたモデルが過去に検出したサンプルの**記憶**を持たない事である．例えば，それらのサンプルはフィードフォワードステップとバックプロパゲーションステップを通過していく．そして，それらの重みは，サンプルが処理される順序とは無関係に更新される．

対照的に，リカレントニューラルネットワーク(RNN)の目的は，シーケンスを設計し，モデル化することにある．RNNは過去の情報を記憶しておき，その情報に従って新しい事象を処理できる．

### シーケンスモデルの様々なカテゴリ

シーケンスモデルには，魅力的な応用例がいくつもある．例えば，言語翻訳，画像キャプショニング，テキスト生成などが挙げられる．

ただし，適切なモデルを開発するには，シーケンスモデルを構築するための様々な種類のタスクを理解する必要がある．

入力データと出力データのどちらかがシーケンスであるとしたら，そのデータは次の3種類のカテゴリのいずれかに分類される．

- **多対一**
  - 入力データはシーケンスだが，出力は(シーケンスではなく)固定サイズのベクトルである．例えば感情分析では，入力はテキストベースであり，出力はクラスラベルである．
- **一対多**
 - 入力データは標準フォーマットであり，シーケンスではないが，出力はシーケンスである．このカテゴリの一例は画像キャプショニングである．画像キャプショニングでは，入力は画像であり，出力は英語のフレーズである．
- **多対多**
 - 入力データと出力データはどちらもシーケンスである．このカテゴリは，入力と出力が同期するかどうかに基づいて更に分類できる．多対多の**同期モデル**の一例は動画分類である．動画分類では，動画の各フレームがラベル付けされる．多対多の**遅延**モデルの一例は言語翻訳である．例えば，英語をドイツ語に翻訳するときには，英語の文章全体を読み込んで処理したうえでドイツ語に翻訳しなければならない．

シーケンスモデルのカテゴリを理解したところで，RNNの構造について見ていく．

## リカレントニューラルネットワーク:シーケンスモデルの構築

### RNNの構造とデータの流れ

標準のフィードフォワードニューラルネットワークでは，情報は入力層から隠れ層へ流れ，隠れ層から出力層へ流れる．これに対し，RNNでは，隠れ層の入力は入力層から得られるだけでなく，**1つ前の時間刻みの隠れ層からも得られる**．

隠れ層の連続する時間刻みの間を情報が流れることにより，ネットワークが過去の事象に対する記憶を持つことが可能になる．こうした情報の流れは，通常はループ(循環)として表示される．グラフ表記ではこのループは**リカレントエッジ**(recurrent edge)とも呼ばれる．このアーキテクチャ全体を「リカレントニューラルネットワーク」と呼ぶのはそのためである．

知っての通り，標準的なニューラルネットワークの隠れユニットが受け取る入力はそれぞれ1つだけである．隠れユニットの入力は，入力層に関連付けられた事前活性化(総入力)である．対照的に，RNNの隠れユニットはそれぞれ，入力層からの事前活性化と，1つ前の時間刻み$t-1$の同じ隠れ層からの活性化という2つの入力を受け取る．

最初の時間刻み$t=0$では，隠れユニットはそれぞれ0または小さな乱数で初期化される．次に，$t\gt0$の時間刻みでは，隠れユニットは2つの場所から入力を受けとる．それらは，現在の時間のデータ点$x^{(t)}$と，1つ前の時間刻み$t-1$の隠れユニットの値$h^{(t-1)}$である．

同様に多層RNNの場合は情報の流れを次のようにまとめることが出来る．

- **layer=1**
 - この場合，隠れ層は$h_{1}^{(t)}$で表される．隠れ層の入力は，データ点$x^{(t)}$と，同じ層の1つ前の時間刻みの隠れユニットの値$h_{1}^{(t-1)}$である．
- **layer=2**
 - 2つ目の隠れ層$h_{2}^{(t)}$は，現在の時間刻みの下にある隠れユニット$h_{1}^{(t)}$と，同じ層の1つ前の時間刻みの隠れユニット$h_{2}^{(t-1)}$から入力を受け取る．

### 長期的な相互作用の学習と勾配消失・発散問題

RNNのトレーニング手法の1つにBPTT(Backpropagation Through Time)がある．これは，損失関数の勾配を計算するときの乗法係数$\frac{\partial h^{(t)}}{\partial h^{(k)}}$により，いわゆる**勾配消失**(vanishing gradient)問題と**勾配発散**(exploding gradient)問題が発生する．

直感的に分かるのは，勾配消失問題や，勾配発散問題を回避するための単純な解決策が，$|w|=1$を保証することによって実現できることである．

実際にはこの問題に対する解決策が2つある．

- T-BPTT(Truncated Backpropagation Through Time)
- 長短期記憶(Long Short-Term Memory:LSTM)

T-BPTTは指定されたしきい値の上で勾配を刈り込む．T-BPTTは勾配発散問題を解決できるが，この刈り込みにより，勾配とは逆方向に進んで重みを正しく更新できる時間刻みの数が制限される．

一方で，LSTMは勾配消失問題を克服することにより，シーケンスでの長期的な依存関係のモデル化において成功を収めている．

以下でLSTMについて少し詳しく見ていく．

### LSTMのユニット

LSTMの構成要素は**メモリセル**(memory cell)である．メモリセルは基本的には隠れ層を表す．

各メモリセルには，勾配消失問題と勾配発散問題を克服するためのリカレントエッジが存在する．

先に述べたように，リカレントエッジの望ましい重みは$w=1$である．このリカレントエッジに関連付けられる値は**セル状態**(cell state)と呼ばれる．次の図は，LSTMセルの構造を展開したものである．

![LSTMセル](images/lstm_cell.jpg)

この図から分かるように，1つ前のセル状態$C^{(t-1)}$が，現在の時間刻みのセル状態を取得するために(重み係数を直接かけることなく)変更される．

このメモリセルでの乗法の流れは，次に説明する計算ユニットによって制御される．

4つのボックスには，活性化関数(シグモイド関数$\sigma$, 双曲線正接関数tanh)といちれんの重みが含まれている．
これらのボックスでは入力で行列とベクトルの乗算を行うことで，線型結合を適用する．

これらの計算つニットとシグモイド活性化関数は**ゲート**(gate)と呼ばれる．ゲートの出力ユニットは$\odot$を通じて渡される．

LSTMセルには，忘却ゲート，入力ゲート，出力ゲートの3種類のゲートが存在する．

- **忘却ゲート**($f_{t}$)では，メモリセルを無限に成長させるのではなく，セル状態をリセットできる．実際には，忘却ゲートは通過させる情報と通過させない情報を決定する．$f_{t}$は次の様に計算される． $$f_{t}=\sigma(W_{xf}x^{(t)}+W_{hf}h^{(t-1)}+b_{f})$$
なお，忘却ゲートは最初からLSTMのセルの一部だったわけではなく，最初のモデルを改善するために数年後に追加されたものである．

- **入力ゲート**($i_{t}$)と入力ノード($g_{t}$)は，セル状態を更新する役割を果たす．入力ゲートと入力ノードは次のように計算される．
$$i_{t}=\sigma(W_{xi}x^{(t)}+W_{hi}h^{(t-1)}+b_{i}) \\ g_{t}=\tanh(W_{xg}x^{(t)}+W_{hg}h^{(t-1)}+b_{g})$$
時間$t$でのセル状態は次のように計算される． $$C^{(t)}=(C^{(t-1)}\odot f_{t})\oplus (i_{t}\odot g_{t})$$

- **出力ゲート**($o_{t}$)は，隠れユニットの値の更新方法を決定する．
$$o_{t}=\sigma(W_{xo}x^{(t)}+W_{ho}h^{(t-1)}+b_{o})$$
したがって，現在の時間刻みでの隠れユニットは次のように計算される．
$$h^{(t)}=o_{t}\odot \tanh(C^{(t)})$$

LSTMセルの構造とそのベースとなる計算は，かなり複雑に思えるかもしれないが，TensorFlowにはLSTMセルを簡単に定義できるラッパー関数が一通り実装されている．

## 多層RNNの実装

### プロジェクト1:IMDb(Internet Movie Database)映画レビューの感情分析

感情分析では，文章またはテキスト文書に表明されている意見を分析する．

ここでは多対一のアーキテクチャに基づいて感情分析をするための多層RNNを実装する．

#### データの準備

映画レビューデータセットはgzipで圧縮されたアーカイブとして[ここ](https://ai.stanford.edu/~amaas/data/sentiment/)で配布されているのでダウンロードして展開しておく．